In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 20160510: both versions of BkAvgAst created (nohist & ultcert)
###           used Tickers from earlier version, then supplemented with missing Tickers if aeTotAst>800bn
### 20170407: updated for 2016-Dec updated FailedBank list and 2017-Mar institutions2.csv file

import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201612.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2016-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2016-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.395950e+09  1.369624e+09  4.808213e+08  6.097738e+08   
1 2001-03-31  1.210100e+09  1.216132e+09  6.332250e+08  7.478789e+08   
2 2001-03-31  1.021323e+09  1.017492e+09  4.930308e+08  2.606809e+08   
3 2001-03-31  7.900139e+08  7.775349e+08  4.884076e+08  5.403720e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.526110e+08  8.623848e+08        NaT  9.155524   9.155524  5.541019   
1  1.083941e+08  8.562730e+08 2009-01-16  7.936602  17.092126  7.297330   
2  4.333959e+08  6.940768e+08 2008-11-23  6.698479  23.790605  5.681723   
3  6.454305e+07  6.049150e+08        NaT  5.181410  28.972015  5.628445   
4  2.469325e+07  2.951088e+08        NaT  2.896596  31.868612  2.934454   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.541019  7.027077   7.027077  
1  12.838349  8.618610  15.645687  
2  18.520072  3.004105  18.649792  
3  24.148517  6.227286  24.877077  
4  27.082971  3.116289  27.993366

In [3]:
len(rk) # 10762

10762

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_201703.csv", encoding='latin-1')
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street     48,570      NM   
1  Connecticut     6       0       0  81 West Main Street    624,655      NM   
2        Maine     8       0       0      One City Center  1,699,404      SM   
3        Maine     9    9781       0       66 Main Street    539,169      SM   
4        Maine    10       0       0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   
2       223         0         0 ...               Portland-South Portland, ME   
3       223         0         0 ...                                       NaN   
4       223         0         0 ...               Portland-South Portland, ME   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   
2    38860               Portland-South Portland, ME      38860   
3        0                                       NaN          0   
4    38860               Portland-South Portland, ME      38860   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  
2              1              0       NaN            0            0 NaN  
3              0              0       NaN            0            0 NaN  
4              1              0       NaN            0            0 NaN  

[5 rows x 132 columns]

In [5]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [6]:
len(bhc) # 27596

27596

In [7]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street   48,570      NM   
1  Connecticut     6       0       0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  

[2 rows x 132 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
len(bhc2) # 27596

27596

In [9]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [10]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635                           AuburnBank  AL      SM      9.0   
19    39  430036            Robertson Banking Company  AL      NM      6.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      2.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA       14  
15        Auburn National Bancorp Inc  1129533.0    AL       35  
19                            Rbc Inc  2526755.0    AL       39  
21       Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Fin Services Inc  1082955.0    AL       46  
26   Traders & Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares IncESOPlan  3587294.0    AL       50  
30        First Vernon Bancshares Inc  1139998.0    AL       51  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL       52  
33                       Gadsden Corp  1079067.0    AL       54

In [11]:
bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

Empty DataFrame
Columns: [Cert, IDRSSD, Name, ST, BkClass, Offices, Parent, ParRSSD, ParST, UltCert]
Index: []

In [12]:
bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [14]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612.csv", index_col=0)
rkhn = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612n.csv", index_col=0)

len(rkh), len(rkhn) # 10762

(10762, 10762)

In [15]:
rkh.columns

Index(['Rank', 'IDRSSD', 'xAsOf', 'xName', 'xState', 'xCert', 'nAsOf',
       'FailDate', 'BkClass', 'Offices', 'Parent', 'ParRSSD', 'ParST',
       'UltCert', 'Ticker', 'aeTotAst', 'aaTotAst', 'aeTotLn', 'aeDDep',
       'aeFDep', 'aeTotDep', 'pr_eTA', 'pr_eTAcs', 'pr_eTL', 'pr_eTLcs',
       'pr_eDD', 'pr_eDDcs'],
      dtype='object')

In [16]:
len(rkh[rkh['Ticker']>'']) # count before new BHCs added

2932

In [17]:
len(rkhn[rkhn['Ticker']>'']) # count after new BHCs added

2932

In [18]:
rkhn[rkhn['Ticker']=='BFIN'] # this will be added by missing ticker file (later)

Rank  IDRSSD       xAsOf                        xName xState    xCert  \
639    640  454676  2016-12-31            BANKFINANCIAL, NA     IL  28808.0   
1490  1491  366434  2001-09-30        SUCCESS NATIONAL BANK     IL  21025.0   
3468  3469  849834  2010-12-31  DOWNERS GROVE NATIONAL BANK     IL  17488.0   
6269  6270  218832  2006-03-31     UNIVERSITY NATIONAL BANK     IL   1793.0   

           nAsOf FailDate BkClass  Offices     ...           aeTotLn  \
639   2011-09-30      NaN       N     20.0     ...      1.165514e+06   
1490  2001-03-31      NaN       N      8.0     ...      4.980437e+05   
3468  2001-03-31      NaN       N      2.0     ...      1.865422e+05   
6269  2001-03-31      NaN       N      2.0     ...      2.077933e+04   

            aeDDep aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
639   1.286427e+06    0.0  1.286427e+06  0.009785  85.761259  0.013431   
1490  4.647363e+05    0.0  4.647363e+05  0.003785  90.666954  0.005739   
3468  2.093832e+05    0.0  2.093832e+05  0.001550  95.370860  0.002150   
6269  9.838843e+04    0.0  9.838843e+04  0.000706  98.320460  0.000239   

       pr_eTLcs    pr_eDD    pr_eDDcs  
639   83.584974  0.014825   86.824169  
1490  89.299623  0.005356   93.596598  
3468  94.787638  0.002413  100.303760  
6269  98.185438  0.001134  104.593451  

[4 rows x 27 columns]

In [19]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkhn2 = rkhn[rkhn['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkhn2) # 8672

8672

In [20]:
rkhn2.tail(2)

Rank    ParRSSD ParST Ticker
10758  10759  1094314.0    MO  CBCYB
10761  10762  3447567.0    TX    NaN

In [21]:
rkh3 = rkhn2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 416

416

In [22]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [23]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [24]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACNB          1
ADKT          1

In [25]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [26]:
bhc2.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [27]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [28]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4706.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [29]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4706.0  Bank Of America Corp  1073757.0    NC     3510

In [30]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eTLcs    pr_eDD  \
0  1.395950e+09  1.369624e+09  4.808213e+08   ...      5.541019  7.027077   
1  1.210100e+09  1.216132e+09  6.332250e+08   ...     12.838349  8.618610   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.027077   628.0        N   5417.0  Jpmorgan Chase & Co.  1039502.0   
1  15.645687  3510.0        N   4706.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [31]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [32]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eDD   pr_eDDcs  \
0  1.395950e+09  1.369624e+09  4.808213e+08   ...    7.027077   7.027077   
1  1.210100e+09  1.216132e+09  6.332250e+08   ...    8.618610  15.645687   

     Cert BkClass  Offices                Parent    ParRSSD  ParST  UltCert  \
0   628.0       N   5417.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0   
1  3510.0       N   4706.0  Bank Of America Corp  1073757.0     NC   3510.0   

   Ticker  
0     JPM  
1     BAC  

[2 rows x 28 columns]

In [33]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [34]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices    ...           aeTotLn        aeDDep  \
0        NaT       N   5417.0    ...      4.808213e+08  6.097738e+08   
1 2009-01-16       N   4706.0    ...      6.332250e+08  7.478789e+08   

         aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
0  2.526110e+08  8.623848e+08  9.155524   9.155524  5.541019   5.541019   
1  1.083941e+08  8.562730e+08  7.936602  17.092126  7.297330  12.838349   

     pr_eDD   pr_eDDcs  
0  7.027077   7.027077  
1  8.618610  15.645687  

[2 rows x 27 columns]

In [35]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201612.csv")

In [36]:
len(rk4) # 10762

10762

In [37]:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 190

190

In [38]:
rkm.head()

Rank  IDRSSD      xAsOf                         xName xState    xCert  \
145   146  527954 2016-12-31  FIRST NATIONAL BANK OF OMAHA     NE   5452.0   
149   150  311845 2016-12-31                   ARVEST BANK     AR   8728.0   
162   163  137915 2016-12-31                 EMIGRANT BANK     NY  12054.0   
182   183  877369 2016-12-31                  RABOBANK, NA     CA  23364.0   
183   184  249612 2016-12-31        APPLE BANK FOR SAVINGS     NY  16068.0   

         nAsOf FailDate BkClass  Offices    ...           aeTotLn  \
145 2001-03-31      NaT       N    126.0    ...      7.053570e+06   
149 2001-03-31      NaT      SM    273.0    ...      6.452287e+06   
162 2001-03-31      NaT      SB      5.0    ...      5.207047e+06   
182 2001-03-31      NaT       N    108.0    ...      5.700340e+06   
183 2001-03-31      NaT      SB     80.0    ...      3.759378e+06   

           aeDDep  aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
145  8.339843e+06     0.0  8.339843e+06  0.069367  73.960962  0.081286   
149  8.516992e+06  3596.0  8.520588e+06  0.066488  74.229182  0.074357   
162  6.415656e+06     0.0  6.415656e+06  0.060657  75.046399  0.060006   
182  5.675783e+06     0.0  5.675783e+06  0.053977  76.185468  0.065691   
183  6.999825e+06     0.0  6.999825e+06  0.053634  76.239102  0.043323   

      pr_eTLcs    pr_eDD   pr_eDDcs  
145  70.212938  0.096109  72.150419  
149  70.521400  0.098150  72.524887  
162  71.280566  0.073934  73.511618  
182  72.635957  0.065408  74.854570  
183  72.679280  0.080666  74.935236  

[5 rows x 27 columns]

In [39]:
rkm.to_csv('rk_missingtickers201612.csv')

In [ ]:
#STOPPED HERE - 20170520